<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 


# Projet de Machine Learning
#### Soutenances orale : 28 Mai 2025

# Introduction

L'analyse des performances physiques et des habitudes d'entraînement est un sujet clé en science du sport et en santé. Dans le cadre de ce projet de **Machine Learning**, nous étudions un jeu de données provenant du site **Kaggle**, intitulé *Gym Members Exercise Dataset* ([source](https://www.kaggle.com/datasets/valakhorasani/gym-members-exercise-dataset)). Ce jeu de données fournit des informations détaillées sur les caractéristiques physiques, les routines d'exercice et la condition cardiovasculaire de **973 membres** fréquentant une salle de sport.

### Objectif de l'étude

L'objectif principal de cette analyse est d'explorer les **facteurs influençant la dépense énergétique et le niveau d'expérience** des individus. Nous nous intéressons en particulier à la **prédiction des calories brûlées** au cours d'une séance d'entraînement ainsi qu'à la **classification du niveau d'expérience des membres** en fonction des autres variables du dataset.

### Approche méthodologique

Dans un premier temps, nous effectuons une **analyse exploratoire** des données, afin de :
- Vérifier la cohérence et la qualité des données.
- Étudier la distribution des variables.
- Identifier les relations entre les variables par des visualisations et des statistiques descriptives.

Ensuite, nous utiliserons des **modèles de Machine Learning** pour prédire :
1. **Les calories brûlées** en fonction des caractéristiques physiques et des habitudes d'entraînement.
2. **Le niveau d'expérience des membres**, en intégrant la dépense calorique comme variable explicative.

Nous comparerons plusieurs algorithmes, tels que la **régression linéaire, les forêts aléatoires, les SVM et les réseaux de neurones**, en optimisant leurs hyperparamètres pour améliorer les performances prédictives.

### Organisation du document

Ce document suit la structure suivante :
1. **Exploration des données et statistiques descriptives**.
2. **Visualisations et corrélations entre variables**.
3. **Modélisation prédictive des calories brûlées et du niveau d’expérience**.
4. **Interprétation des résultats et conclusions**.

Cette analyse permettra de mieux comprendre les **facteurs influençant la performance et l'expérience sportive**, avec des applications potentielles en coaching sportif et en suivi de la condition physique.

---


# Statistiques descriptives 

In [2]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)

## Prise en charge des données

Le jeu de données contient **15 variables**, qui peuvent être classées en différentes catégories :

- **Données démographiques :**  
  - `Age` : âge du membre (en années).  
  - `Gender` : sexe du membre (*Male* ou *Female*).  
  - `Weight (kg)`, `Height (m)`, `BMI` : caractéristiques anthropométriques.  
  - `Fat_Percentage` : pourcentage de graisse corporelle.  

- **Données cardiovasculaires :**  
  - `Max_BPM`, `Avg_BPM`, `Resting_BPM` : mesures de la fréquence cardiaque avant et pendant l'entraînement.  

- **Informations sur l'entraînement :**  
  - `Workout_Type` : type d'entraînement (Cardio, Musculation, Yoga, HIIT).  
  - `Session_Duration (hours)`, `Calories_Burned` : durée et intensité de la séance.  
  - `Workout_Frequency (days/week)`, `Experience_Level` : fréquence et niveau d’expérience (1 = Débutant, 3 = Expert).  

- **Facteurs de mode de vie :**  
  - `Water_Intake (liters)` : consommation d’eau quotidienne.  

L'**indice de masse corporelle (IMC)** est défini par la formule :

$$ BMI = \frac{\text{Poids (kg)}}{\text{Taille (m)}^2} $$

In [5]:
# Lecture des données
# path="http://www.math.univ-toulouse.fr/~besse/Wikistat/data/"
path <- ""
ozone <- read.table(paste(path, "gym_members_exercise_tracking.csv", sep = ""),
                    sep = ",", header = TRUE)
# Premières lignes du jeu de données
head(ozone)
# Vérification du contenu
summary(ozone)

,Age,Gender,Weight..kg.,Height..m.,Max_BPM,Avg_BPM,Resting_BPM,Session_Duration..hours.,Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake..liters.,Workout_Frequency..days.week.,Experience_Level,BMI
,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,56,Male,88.3,1.71,180,157,60,1.69,1313,Yoga,12.6,3.5,4,3,30.20
2,46,Female,74.9,1.53,179,151,66,1.30,883,HIIT,33.9,2.1,4,2,32.00
3,32,Female,68.1,1.66,167,122,54,1.11,677,Cardio,33.4,2.3,4,2,24.71
4,25,Male,53.2,1.70,190,164,56,0.59,532,Strength,28.8,2.1,3,1,18.41
5,38,Male,46.1,1.79,188,158,68,0.64,556,Strength,29.2,2.8,3,1,14.39
6,56,Female,58.0,1.68,168,156,74,1.59,1116,HIIT,15.5,2.7,5,3,20.55


      Age           Gender           Weight..kg.       Height..m.   
 Min.   :18.00   Length:973         Min.   : 40.00   Min.   :1.500  
 1st Qu.:28.00   Class :character   1st Qu.: 58.10   1st Qu.:1.620  
 Median :40.00   Mode  :character   Median : 70.00   Median :1.710  
 Mean   :38.68                      Mean   : 73.85   Mean   :1.723  
 3rd Qu.:49.00                      3rd Qu.: 86.00   3rd Qu.:1.800  
 Max.   :59.00                      Max.   :129.90   Max.   :2.000  
    Max_BPM         Avg_BPM       Resting_BPM    Session_Duration..hours.
 Min.   :160.0   Min.   :120.0   Min.   :50.00   Min.   :0.500           
 1st Qu.:170.0   1st Qu.:131.0   1st Qu.:56.00   1st Qu.:1.040           
 Median :180.0   Median :143.0   Median :62.00   Median :1.260           
 Mean   :179.9   Mean   :143.8   Mean   :62.22   Mean   :1.256           
 3rd Qu.:190.0   3rd Qu.:156.0   3rd Qu.:68.00   3rd Qu.:1.460           
 Max.   :199.0   Max.   :169.0   Max.   :74.00   Max.   :2.000           